# Un WebSocket ECHO Serveur écrit en Rust

---

## Introduction

Dans cette partie, nous allons réaliser un simple echo serveur qui utilise les websockets, pour ce faire, j'ai choisi d'utiliser la crate `ws` qui semble être en mesure de pouvoir gérer cette tâche.

> L'objectif de ce chapitre est de présenter les différents raisonnements qui seraient nécessaires à l'implémentation d'une crate que l'on ne connaissait pas jusqu'alors.

**Pour l'implémentation cote serveur, nous irons chercher la crates ws depuis crates.io, le fichier cargo.toml rassemblera à cela :**

```
[package]
name = "websocket-server"
version = "0.1.0"
edition = "2021"

# See more keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html

[dependencies]
ws = "0.9.2" 
```

![WS](./pictures/ws-rs.png)

## Se documenter

- Une fois la crate mise dans les dependances du projet, nous lanceons de suite la documentation du projet avec `cargo doc --open` afin d'obtenir des informations sur comment utiliser la crate websocket :

![DOC!](./pictures/doc-page1.png)

*Nous voyons ici la documentation de la crate principal, celle de notre main.rs, il n'y a rien pour l'instant !*

- Dans la partie crates à gauche, allons vers la documentation de ws :

![DOCU2](./pictures/doc-page2.png)

*Aucun exemple n'est présente hélas, mais nous remarquons la présence d'une structure `WebSocket`, d'un trait `Handler` qui semble intéressant et aussi 2 fonctions `connect` et `listen`, qui semblent pouvoir faire le café...*

- Comme nous sommes sur un programme server, il est évident que c'est la fonction `listen` que nous allons utiliser, allons-y :

```
Function ws::listen

pub fn listen<A, F, H>(addr: A, factory: F) -> Result<()>where
    A: ToSocketAddrs + Debug,
    F: FnMut(Sender) -> H,
    H: Handler,

A utility function for setting up a WebSocket server.
Safety

This function blocks until the event loop finishes running. Avoid calling this method within another WebSocket handler.
Examples

use ws::listen;

listen("127.0.0.1:3012", |out| {
    move |msg| {
       out.send(msg)
   }
}).unwrap()
```

> Un exemple est présent, la fonction semble s'exécuter sans nécessiter de préalables, le mieux à faire ici est donc de copier/coller l'exemple dans notre fichier main.rs.

**À la place de l'IP/PORT propose, il est préférable de modifier en `0.0.0.0` afin de pouvoir écouter sur TOUTES les interfaces, et sélectionner un port valable pour l'image Docker ou notre OS. (8081-8089 pour Docker et +1000 pour OS).**

- On développe aussi un peu avec les brackets afin que le code paraisse plus clair :

```
use ws::listen;

fn main() {
    listen("0.0.0.0:8800", |out| {
        println!("Inside first closure");
        move |msg| {
            println!("Inside second closure");
            out.send(msg)
        }
    })
    .expect("Cannot Listen");
} 
```

- `F: FnMut(Sender) -> H,` indique la nécessité de mettre en closure, le type `Sender` désigne-t-il l'envoyer du paquet ? La closure semble retourner un autre générique `H` ...

```
    |out| {
        println!("Inside first closure");
        move |msg| {
            println!("Inside second closure");
            out.send(msg)
        }
    }
```

- `H: Handler,` désigne un trait que doit implémenter le type de retour de la closure, implémenter Handler pour un type permet d'implémenter toute une liste de méthodes relatives aux websockets :

```
        move |msg| {
            println!("Inside second closure");
            out.send(msg)
        }
```

*Ici, ça devient étrange, on a du mal à imaginer que cette closure implémente le trait Handler...*

```
pub trait Handler {
    // Provided methods
    fn on_shutdown(&mut self) { ... }
    fn on_open(&mut self, shake: Handshake) -> Result<()> { ... }
    fn on_message(&mut self, msg: Message) -> Result<()> { ... }
    fn on_close(&mut self, code: CloseCode, reason: &str) { ... }
    fn on_error(&mut self, err: Error) { ... }
    fn on_request(&mut self, req: &Request) -> Result<Response> { ... }
    fn on_response(&mut self, res: &Response) -> Result<()> { ... }
    fn on_timeout(&mut self, event: Token) -> Result<()> { ... }
    fn on_new_timeout(&mut self, _: Token, _: Timeout) -> Result<()> { ... }
    fn on_frame(&mut self, frame: Frame) -> Result<Option<Frame>> { ... }
    fn on_send_frame(&mut self, frame: Frame) -> Result<Option<Frame>> { ... }
    fn build_request(&mut self, url: &Url) -> Result<Request> { ... }
}
```
*Notez que ce sont des `Provided methods` et non des `Required Methods`, donc il est tout a fait possible de ne pas en implémenter certaines parce que non utile dans notre cas.*

## Refactorisation de l'exemple

> Quelle proposition de code pouvez-vous faire dans le main afin d'avoir le code le plus simple possible ?

### Proposition de code

- Après réflexion, il semble que la forme la plus simple de l'utilisation de listen() semble de la sorte :

```
    struct Instance {}

    impl ws::Handler for Instance;
    
    listen("0.0.0.0:8800", |out: ws::Sender| -> Instance {
        println!("Inside first closure");
        Instance {}
    })
    .expect("Cannot Listen");  
```

Mais si cette forme-là est juste, comment se fait-il que dans l'exemple proposé par la documentation, il n'y avait aucun type de retour de la closure alors que cette même documentation indique qu'**un type de retour doit implémenter le trait Handler** ??? 

>```
>use ws::listen;
>
>listen("127.0.0.1:3012", |out| { // Il n'y a aucun type de retour pour cette closure, wtf !
>    move |msg| {
>       out.send(msg)
>   }
>}).unwrap()
>```

Dans la documentation du trait Handler, il y a tout en bas dans la catégorie Implementors cela :
```
Implementors

impl<F> Handler for F
where
    F: Fn(Message) -> Result<()>,
```

Comprenez-vous mieux maintenant ? C'est une implémentation générique pour tout type de fonction ou de closures dont le prototype est `Fn(Message) -> Result<()>`, c'est le cas dans l'exemple de la documentation. **C'est très velu, car il faut aller chercher tout en bas d'une page de documentation d'une crate que l'on découvre tout juste pour comprendre cette magie !**

### Le paramètre out

- Reprenons le code que nous avons proposé plus haut :

```
    struct Instance {}

    impl ws::Handler for Instance;

    listen("0.0.0.0:8800", |out: ws::Sender| -> Instance {
        println!("Inside first closure");
        Instance {}
    })
    .expect("Cannot Listen");  
```
Ce code compile très bien et peut se lancer, mais un warning nous indique que l'on ne fait rien de la variable `out` qui est de type `Sender`. Comme son nom l'indique, cette variable est de type Sender qui permet **d'envoyer des données** au client, et de l'autre cote, l'implémentation du trait Handler que l'on fera pour Instance semble plutôt gérer la partie de l'**écoute du client**.

## Les méthodes de l'implémentation du trait Handler

**Si on regarde maintenant les différentes méthodes du trait Handler, on peut grosso modo les classer en 4 catégories.**

### Les méthodes de changement d'état du socket

>- on_shutdown
>- on_open
>- on_close
>- on_error

### La méthode de réception standard

> - on_message

### Des méthodes plus bas niveau

> - on_frame
> - on_send_frame

### Des méthodes "spécialisées"

> - on_timeout
> - on_new_timeout
> - on_request
> - on_response
> - build_request

**Je vous propose d'implémenter les méthodes de changement d'état en y mettant des println! afin que vous puissiez voir ce qui se passe. Et d'écrire aussi un println! qui lirait le message reçu par la socket pour on_message.**

## Un dernier élément

**Si l'on veut terminer notre ECHO SERVER, quelle partie de code manque-t-il ?**

*(pas de solution disponible ici)*

## Conclusion

Dans cette partie, nous venons de réaliser un echo server en Rust se servant de l'api websocket, si je lance le serveur, la commande `netstat -antup` me retourne l'information que le programme écoute :

`tcp        0      0 0.0.0.0:8800            0.0.0.0:*               LISTEN      13428/target/debug/`

Dans la partie suivante, nous développerons la cote client en web assembleur avec les websockets de l'api WEB.